In [1]:
import spark_catalog
import duckdb

In [2]:
catalog = spark_catalog.SparkCatalog(app_name="kexp", lake_location="../../../data/spark/kexp")

:: loading settings :: url = jar:file:/Users/timburns/Documents/PythonProjects/.pyenv/kexp/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/timburns/.ivy2/cache
The jars for the packages stored in: /Users/timburns/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-6b85d48d-40a0-4bf6-89b2-3a6201e461b5;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.3.0 in central
	found io.delta#delta-storage;2.3.0 in central
	found org.antlr#antlr4-runtime;4.8 in central
:: resolution report :: resolve 134ms :: artifacts dl 6ms
	:: modules in use:
	io.delta#delta-core_2.12;2.3.0 from central in [default]
	io.delta#delta-storage;2.3.0 from central in [default]
	org.antlr#antlr4-runtime;4.8 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0  

23/04/09 20:28:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
/Users/timburns/Documents/PythonProjects/.pyenv/kexp/lib/python3.9/site-packages/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [3]:
kexp_playlist = catalog.get_data_frame("bronze", "KEXP_PLAYLIST")

In [4]:
kexp_playlist.show()

23/04/09 20:29:07 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------------------+--------------------+--------------------+--------------------+--------------------+-------+--------------------+-------+--------+----------+--------------------+--------------------+---------+--------------------+------------+--------------------+--------------------+---------------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
|             airdate|               album|              artist|          artist_ids|             comment|     id|           image_uri|is_live|is_local|is_request|           label_ids|              labels|play_type|        recording_id|release_date|    release_group_id|          release_id|rotation_status| show|            show_uri|                song|       thumbnail_uri|            track_id|                 uri|
+--------------------+--------------------+--------------------+--------------------+--------------------+-------+--------------------+-------+--------+----------

In [7]:
catalog.sql("select count(*) as play_count from KEXP_PLAYLIST").show()

+----------+
|play_count|
+----------+
|       863|
+----------+



In [13]:
catalog.sql("select id, airdate, artist, song, show from KEXP_PLAYLIST order by id desc").show()

+-------+--------------------+--------------------+--------------------+-----+
|     id|             airdate|              artist|                song| show|
+-------+--------------------+--------------------+--------------------+-----+
|3178537|2023-04-10T06:11:...|               M.E.B|    Bitches Are Back|56663|
|3178536|2023-04-10T06:08:...|El Michels Affair...|            Grateful|56663|
|3178535|2023-04-10T06:03:...|        Public Enemy|Welcome to the Te...|56663|
|3178534|2023-04-10T06:03:...|                null|                null|56663|
|3178533|2023-04-10T05:59:...|Duval Timothy fea...|                 Kam|56663|
|3178532|2023-04-10T05:56:...|Petite Noir feat....|       Simple Things|56663|
|3178531|2023-04-10T05:52:...|Everything Is Rec...| Close But Not Quite|56663|
|3178530|2023-04-10T05:49:...|     Curtis Mayfield|  The Makings of You|56663|
|3178528|2023-04-10T05:46:...|         Brother Ali|      Dear Black Son|56663|
|3178527|2023-04-10T05:43:...|         Brother Ali| 

In [14]:
catalog.get_data_frame("bronze", "KEXP_PLAYLIST").printSchema()

root
 |-- airdate: string (nullable = true)
 |-- album: string (nullable = true)
 |-- artist: string (nullable = true)
 |-- artist_ids: string (nullable = true)
 |-- comment: string (nullable = true)
 |-- id: string (nullable = true)
 |-- image_uri: string (nullable = true)
 |-- is_live: string (nullable = true)
 |-- is_local: string (nullable = true)
 |-- is_request: string (nullable = true)
 |-- label_ids: string (nullable = true)
 |-- labels: string (nullable = true)
 |-- play_type: string (nullable = true)
 |-- recording_id: string (nullable = true)
 |-- release_date: string (nullable = true)
 |-- release_group_id: string (nullable = true)
 |-- release_id: string (nullable = true)
 |-- rotation_status: string (nullable = true)
 |-- show: string (nullable = true)
 |-- show_uri: string (nullable = true)
 |-- song: string (nullable = true)
 |-- thumbnail_uri: string (nullable = true)
 |-- track_id: string (nullable = true)
 |-- uri: string (nullable = true)

